<a href="https://colab.research.google.com/github/saranshikens/Epoch-Spring-Camp/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPLEMENTING DECISION TREE FROM SCRATCH**  
By - Saransh

In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

**ENCODING THE LABELS**

In [ ]:
encoded_data = [row[:] for row in data] # Create a copy of data. Encoding will be done with this list, so that 'data' remains preserved.
for row in encoded_data:
  if(row[3]=='Wine'): row[3]=0
  elif(row[3]=='Beer'): row[3]=1
  else: row[3]=2

**CREATING THE FEATURE MATRIX AND LABEL VECTOR**

In [ ]:
features = [column[:3] for column in encoded_data] # Extract first three columns from 'encoded_data' and store them as a matrix i.e. a nested list
X = np.array(features)
label = [column[3] for column in encoded_data] # Extract the fourth column from 'encoded_data'
y = np.array(label)

**GINI IMPURITY**  
Calculating Gini Impurity for a particular Attribute with a predecided threshold.  
From here on,  
attributeID = $\begin{cases}
 0, \text{ iff attribute = Alcohol Content}\\
 1, \text{ iff attribute = Sugar Content}\\
 2, \text{ iff atttribute = Color}
\end{cases}$


In [ ]:
def GiniImpurity(attributeID, threshold):
  leftCnt = rightCnt = 0;
  for i in range(len(X)):
    if(X[i][attributeID] <= threshold): leftCnt+=1
    else: rightCnt+=1
  gini = 1 - (leftCnt/len(X))**2 - (rightCnt/len(X))**2
  return gini

Determining the most optimal label (and threshold) for the root node.

In [ ]:
def LeastGiniImpurity(X, y):
  minGini = 2 # Gini impurity lies between 0 and 1, hence this initialization of minGini
  for attribute in range(len(X[0])):
    for threshold in range(len(X)):
      if(GiniImpurity(attribute, threshold) < minGini):
        minGini = GiniImpurity(attribute, threshold)
        minGiniAttribute = attribute
        minGiniThreshold = X[threshold][attribute]
  result = []
  result.append(minGiniAttribute)
  result.append(minGiniThreshold)
  return result

**ENTROPY**  
An alternative to Gini Impurity to compute Impurity

In [ ]:
def entropy(attributeID, threshold):
  leftCnt = rightCnt = 0;
  for i in range(len(X)):
    if(X[i][attributeID] <= threshold): leftCnt+=1
    else: rightCnt+=1
  leftProb = leftCnt/len(X)
  rightProb = rightCnt/len(X)
  entropy = -1 * (leftProb * np.log2(leftProb) + rightProb * np.log2(rightProb))
  return entropy

Determining the most optimal label (and threshold) for the root node.

In [ ]:
def LeastEntropy(X, y):
  minEntropy = 2 # Entropy lies between 0 and 1, hence this initialization of minEntropy
  for attribute in range(len(X[0])):
    for threshold in range(len(X)):
      if(entropy(attribute, threshold) < minEntropy):
        minEntropy = entropy(attribute, threshold)
        minEntropyAttribute = attribute
        minEntropyThreshold = X[threshold][attribute]
  result = []
  result.append(minEntropyAttribute)
  result.append(minEntropyThreshold)
  return result

Determining the entry under an attribute, with max occurence

In [ ]:
def majority_class(attributes): # I ChatGPT'ed this thing
  return Counter(attributes).most_common(1)[0][0]

**RECURSIVE TREE BUILDING**

In [ ]:
class Node:
  def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def buildTree(X, y, depth=0, maxDepth=5):
    if(len(set(y))==1 or depth >= maxDepth):
      leaf = majority_class(y) if len(y)>0 else None
      return Node(value=leaf)

    attributeID, threshold = LeastEntropy(X, y) # We can hard-code the impurity measure type here.

    leftIDs = [i for i in range(len(X)) if X[i][attributeID] <= threshold]
    rightIDs = [i for i in range(len(X)) if X[i][attributeID] > threshold]

    # Splitting X and y for left and right nodes
    left = Node.buildTree(X[leftIDs], y[leftIDs], depth+1, maxDepth) if len(leftIDs) > 0 else None
    right = Node.buildTree(X[rightIDs], y[rightIDs], depth+1, maxDepth) if len(rightIDs) > 0 else None

    return Node(attributeID, threshold, left, right)

  def predict_one(self, x):
    if self.value is not None:
        return self.value
    if x[self.feature_index] <= self.threshold:
        return self.left.predict_one(x)
    else:
        return self.right.predict_one(x)

  def predict(tree, X_test):
    return [tree.predict_one(x) for x in X_test]


**TESTING**

In [ ]:
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])


In [ ]:
tree = Node.buildTree(X, y)
predictions = Node.predict(tree, test_data)
for i in range(len(predictions)):
  if(predictions[i]==0): predictions[i]='Wine'
  elif (predictions[i]==1): predictions[i]='Beer'
  else: predictions[i]='Whiskey'
print(predictions)

['Beer', 'Wine', 'Wine']


<ipython-input-45-01ad8639fa17>:8: RuntimeWarning: divide by zero encountered in log2
  entropy = -1 * (leftProb * np.log2(leftProb) + rightProb * np.log2(rightProb))
<ipython-input-45-01ad8639fa17>:8: RuntimeWarning: invalid value encountered in scalar multiply
  entropy = -1 * (leftProb * np.log2(leftProb) + rightProb * np.log2(rightProb))
